<a href="https://colab.research.google.com/github/seodaegal/Textmining/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%20%EB%89%B4%EC%8A%A4%20GPT%20%ED%86%A0%ED%94%BD%20%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

4주차 과제는 네이버 뉴스 GPT 토픽 모델링입니다.

1. 신규 용어가 많이 등장하므로 사용자 사전을 구축합니다.
2. 토픽 모델링과 토픽에 대한 해석

위의 두가지 과정에서 여러분들의 분석 과정의 깊이를 보여주시기를 부탁드립니다. 전처리 방법과 토픽 개수 지정 등 다양하게 활용해보시기 바랍니다.

In [ ]:
# pyLDAvis 설치
! pip install pyLDAvis

In [ ]:
# 한글 폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [ ]:
!bash install_mecab-ko_on_colab_light_220429.sh

In [ ]:
cd /content/mecab-ko-dic-2.1.1-20180720

/content/mecab-ko-dic-2.1.1-20180720


In [ ]:
!pip install jamo

from jamo import h2j, j2hcj
def get_jongsung_TF(sample_text):
    sample_text_list = list(sample_text)
    last_word = sample_text_list[-1]
    last_word_jamo_list = list(j2hcj(h2j(last_word)))
    last_jamo = last_word_jamo_list[-1]

    jongsung_TF = "T"

    if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']:
        jongsung_TF = "F"

    return jongsung_TF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def make_user_dic_csv(morpheme_type, word_list, user_dic_file_name):
  file_data = []

  for word, score in word_list:
    jongsung_TF = get_jongsung_TF(word)

    line = f"{word},,,{score},{morpheme_type},*,{jongsung_TF},{word},*,*,*,*,*\n"

    file_data.append(line)

  with open("./user-dic/user-nnp.csv", 'w', encoding='utf-8') as f:
    for line in file_data:
      f.write(line)

# 네이버 뉴스

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os
path = '/content/drive/MyDrive/딥다이브/4강'
filename = '네이버 뉴스_2023 GPT.csv'

df = pd.read_csv(os.path.join(path, filename))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36011 entries, 0 to 36010
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   제목         36011 non-null  object
 1   언론사        36011 non-null  object
 2   날짜         36011 non-null  object
 3   URL        36011 non-null  object
 4   네이버뉴스_URL  16057 non-null  object
dtypes: object(5)
memory usage: 1.4+ MB


In [ ]:
df.head()

,제목,언론사,날짜,URL,네이버뉴스_URL
0,[신년사] 윤주환 / 한국물산업협의회 회장(고려대 명예교수),에너지데일리,2023.01.01.,http://www.energydaily.co.kr/news/articleView....,NaN
1,[이학무의 테크리딩] AI 성공 키워드는 산업별 최적화한 경험과 지식,IT조선,2023.01.01.,https://it.chosun.com/site/data/html_dir/2022/...,NaN
2,"세계가 열광하는 챗GPT, 수익 창출에도 성공할 수 있을까?",블로터언론사 선정,2023.01.01.,http://www.bloter.net/newsView/blt202212300021,https://n.news.naver.com/mnews/article/293/000...
3,[신년특집/다시 부는 AI 열풍①] 왜 생성 AI인가?...새해 테크판 최대 화두,디지털투데이,2023.01.01.,http://www.digitaltoday.co.kr/news/articleView...,NaN
4,AI로 명품 사로잡은 마크비전코리아의 비결,주간조선언론사 선정,2023.01.01.,http://weekly.chosun.com/news/articleView.html...,https://n.news.naver.com/mnews/article/053/000...


In [ ]:
# URL 기준 중복값 확인
df[df.duplicated(subset = 'URL')]

,제목,언론사,날짜,URL,네이버뉴스_URL
197,"유영상 SKT 사장, CES서 글로벌 연대 강화",이코노믹리뷰,2023.01.08.,https://www.econovill.com/news/articleView.htm...,NaN
1050,'인력난' 코딩 기술자 없어도 IT사업…서비스형SW 주목,뉴시스언론사 선정,2023.01.25.,http://www.newsis.com/view/?id=NISX20230124_00...,https://n.news.naver.com/mnews/article/003/001...
1213,"AI 일상화 '성큼'…7100억원 투입해 ""AI 초일류 강국으로 도약""",뉴스1,2023.01.26.,https://www.news1.kr/articles/4934497,https://n.news.naver.com/mnews/article/421/000...
1214,국가데이터 정책 논의하는 한덕수 총리,뉴스1,2023.01.26.,https://www.news1.kr/photos/view/?5805983,https://n.news.naver.com/mnews/article/421/000...
1215,국가데이터정책위 주재하는 한덕수 총리,뉴스1,2023.01.26.,https://www.news1.kr/photos/view/?5805990,https://n.news.naver.com/mnews/article/421/000...
...,...,...,...,...,...
34567,"IBK기업은행, '한화그룹 우수협력사 일자리박람회' 개최",뉴스워커,2023.05.02.,http://www.newsworker.co.kr/news/articleView.h...,NaN
34677,"머스크 ""AI에 대한 선의의 의존도 문명에 위협"" 또 경고",머니투데이,2023.05.02.,http://news.mt.co.kr/mtview.php?no=20230502162...,https://n.news.naver.com/mnews/article/008/000...
34977,"바티AI, ‘2023년 비대면 스타트업 육성사업’ 선정",세계비즈,2023.05.03.,http://www.segyebiz.com/newsView/2023050251205...,NaN
35671,"고양시, 국토부 ‘2023년 거점형 스마트시티 조성사업’ 최종 선정돼",메트로신문,2023.05.04.,http://www.metroseoul.co.kr/article/2023050450...,NaN


In [ ]:
# URL 기준 중복값 제거
df.drop_duplicates(subset = 'URL', keep = 'first', inplace = True)

In [ ]:
# 결측치 제거
df.dropna(inplace = True)

In [ ]:
# 추가한 사용자 사전이 잘 작동하는지 확인
from konlpy.tag import Mecab

m = Mecab(dicpath = '/content/mecab-ko-dic-2.1.1-20180720')

In [ ]:
# 본문 전처리
import re

df['제목_전처리'] = df['제목'].map(lambda x : re.sub(r'[^\s\w]', ' ', x)) # Text Cleaning
df['제목_전처리'] = df['제목'].map(lambda x : m.nouns(x)) # Tokenizing

In [ ]:
# 본문 전처리 내용 확인
df['제목_전처리']

2                     [세계, 열광, 챗, 수익, 창출, 성공, 수]
4                          [명품, 마크, 비전, 코리아, 비결]
5                 [챗, 봇, 정답, 美, 신종, 컨, 닝, 법, 유행]
7               [고독사, 해법, 지역, 사회, 전문가, 처방, 일맥상통]
8          [소설, 기사, 시나리오, 논문, 수, 생성, 합리, 결정, 도움]
                          ...                   
36004                           [구글, 제국, 재건, 해법]
36007           [포커스, 시대, 도래, 편리, 보안, 우려, 양날, 검]
36008    [시민, 기자, 북, 클럽, 리비, 눈, 최대, 개, 이게, 말, 것]
36009                    [챗, 단속, 나선, 기업, 보안, 효율]
36010           [과학, 스푼, 사람, 생각, 마비, 환자, 소통, 가능]
Name: 제목_전처리, Length: 16006, dtype: object

In [ ]:
# 단어 리스트 만들기
word_list = df['제목_전처리'].explode() # .explode() 함수를 사용하면 리스트 안에 리스트를 폭파(!)시킬 수 있습니다.
print(word_list)


2        세계
2        열광
2         챗
2        수익
2        창출
         ..
36010    생각
36010    마비
36010    환자
36010    소통
36010    가능
Name: 제목_전처리, Length: 118123, dtype: object


In [ ]:
# 단어 빈도 분석
from collections import Counter
c = Counter(word_list)
c.most_common(100)

[('챗', 6486),
 ('봇', 869),
 ('구글', 866),
 ('활용', 715),
 ('반도체', 699),
 ('인공지능', 676),
 ('기술', 656),
 ('기업', 649),
 ('개발', 648),
 ('출시', 638),
 ('만', 571),
 ('서비스', 565),
 ('투자', 562),
 ('것', 548),
 ('생성', 508),
 ('시대', 508),
 ('일', 489),
 ('등', 488),
 ('교육', 485),
 ('년', 480),
 ('美', 467),
 ('디지털', 443),
 ('한국', 438),
 ('글로벌', 425),
 ('미래', 425),
 ('데이터', 423),
 ('공개', 423),
 ('오픈', 418),
 ('주', 412),
 ('검색', 409),
 ('네이버', 405),
 ('카카오', 377),
 ('열풍', 375),
 ('억', 375),
 ('월', 367),
 ('원', 347),
 ('명', 342),
 ('세계', 341),
 ('거대', 340),
 ('테크', 335),
 ('정보', 328),
 ('시장', 327),
 ('혁신', 317),
 ('삼성', 311),
 ('개최', 307),
 ('조', 289),
 ('정부', 287),
 ('수', 284),
 ('경쟁', 276),
 ('산업', 266),
 ('中', 261),
 ('중', 260),
 ('지원', 260),
 ('사업', 252),
 ('규제', 246),
 ('클라우드', 244),
 ('빅', 244),
 ('인간', 237),
 ('스타트업', 235),
 ('대화', 234),
 ('증시', 234),
 ('도입', 232),
 ('폼', 230),
 ('머스크', 229),
 ('메타', 227),
 ('위', 222),
 ('주가', 222),
 ('플랫', 222),
 ('국내', 221),
 ('추진', 212),
 ('기반', 211),
 ('뉴스', 206

In [ ]:
# 단어들이 어떤 맥락에서 사용되는지 알아보기
search_word = '챗'
df[df['제목_전처리'].map(lambda x : search_word in x)]

,제목,언론사,날짜,URL,네이버뉴스_URL,제목_전처리
2,"세계가 열광하는 챗GPT, 수익 창출에도 성공할 수 있을까?",블로터언론사 선정,2023.01.01.,http://www.bloter.net/newsView/blt202212300021,https://n.news.naver.com/mnews/article/293/000...,"[세계, 열광, 챗, 수익, 창출, 성공, 수]"
5,"[Pick] ""AI 챗봇, 정답을 알려줘""…美 신종 컨닝법 유행에 '떠들석'",SBS언론사 선정,2023.01.01.,https://news.sbs.co.kr/news/endPage.do?news_id...,https://n.news.naver.com/mnews/article/055/000...,"[챗, 봇, 정답, 美, 신종, 컨, 닝, 법, 유행]"
12,1초가 다르게 똑똑해지는 챗GPT…‘오케이 구글’ 시대 끝나나,한겨레,2023.01.02.,https://www.hani.co.kr/arti/economy/it/1074015...,https://n.news.naver.com/mnews/article/028/000...,"[초, 챗, 구글, 시대, 끝, 나나]"
17,구글 시대 끝났다? 신개념 대화형 인공지능 ‘챗GPT’ 등장,주간동아언론사 선정,2023.01.02.,https://weekly.donga.com/3/all/11/3860233/1,https://n.news.naver.com/mnews/article/037/000...,"[구글, 시대, 끝, 개념, 대화, 인공지능, 챗, 등장]"
20,"英 오프퀄 ""챗GPT로 컨닝시 실격""...지침 논의 중",ZDNet Korea언론사 선정,2023.01.02.,https://zdnet.co.kr/view/?no=20230102085958,https://n.news.naver.com/mnews/article/092/000...,"[英, 오프, 챗, 컨, 닝, 시, 실격, 지침, 논의, 중]"
...,...,...,...,...,...,...
35967,해커도 챗GPT 쓴다…양날의 칼 AI [AI토피아],서울경제,2023.05.06.,https://www.sedaily.com/NewsView/29PH1OQPI3,https://n.news.naver.com/mnews/article/011/000...,"[해커, 챗, 양날, 칼, 토피아]"
35972,챗GPT에 부동산 투자 조언 구해보니…,주간동아언론사 선정,2023.05.06.,https://weekly.donga.com/3/all/11/4133146/1,https://n.news.naver.com/mnews/article/037/000...,"[챗, 부동산, 투자, 조언]"
35990,"챗GPT, 내 정보도 학습했을까[궁금한AI]",이데일리언론사 선정,2023.05.06.,http://www.edaily.co.kr/news/newspath.asp?news...,https://n.news.naver.com/mnews/article/018/000...,"[챗, 정보, 학습]"
35992,"구글, 검색엔진에 AI 챗봇·동영상 대폭 강화한다…다음주 ‘마기 프로젝트’ 공개 전망",조선일보,2023.05.06.,https://www.chosun.com/economy/economy_general...,https://n.news.naver.com/mnews/article/023/000...,"[구글, 검색, 엔진, 챗, 봇, 동영상, 강화, 다음, 주, 마기, 프로젝트, 공..."


다시 사용자 사전 추가

In [ ]:
word_list = [('챗GPT',0), ('챗지피티',0),('ChatGPT', 0), ('챗 지피티',0)]

make_user_dic_csv(morpheme_type="NNP", word_list=word_list, user_dic_file_name='user-nnp.csv')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!bash autogen.sh

In [ ]:
!bash ./tools/add-userdic.sh

In [ ]:
!make

In [ ]:
!sudo make install
!bash tools/add-userdic.sh

In [ ]:
# Mecab 라이브러리 다시 불러오기
from konlpy.tag import Mecab
m = Mecab(dicpath = '/content/mecab-ko-dic-2.1.1-20180720')

In [ ]:
# 다시 본문 전처리
df['제목_전처리'] = df['제목'].map(lambda x : re.sub(r'[^\s\w]', ' ', x)) # Text Cleaning
df['제목_전처리'] = df['제목_전처리'].map(lambda x: m.nouns(x)) # 명사 기준 토큰화

In [ ]:
# 다시 전처리 결과 확인
df['제목_전처리']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2                      [세계, 열광, 챗, 수익, 창출, 성공, 수]
4                           [명품, 마크, 비전, 코리아, 비결]
5                  [챗, 봇, 정답, 美, 신종, 컨, 닝, 법, 유행]
7                [고독사, 해법, 지역, 사회, 전문가, 처방, 일맥상통]
8           [소설, 기사, 시나리오, 논문, 수, 생성, 합리, 결정, 도움]
                           ...                   
36004                                [구글, 제국, 해법]
36007            [포커스, 시대, 도래, 편리, 보안, 우려, 양날, 검]
36008    [시민, 기자, 북, 클럽, 가리비, 눈, 최대, 개, 이게, 말, 것]
36009                     [챗, 단속, 나선, 기업, 보안, 효율]
36010            [과학, 스푼, 사람, 생각, 마비, 환자, 소통, 가능]
Name: 제목_전처리, Length: 16006, dtype: object

In [ ]:
# 불용어 제거: 한 글자 단어 처리
df['제목_전처리'] = df['제목_전처리'].map(lambda x :
                                [word for word in x if len(word) != 1])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
word_list = df['제목_전처리'].explode()

In [ ]:
from collections import Counter
c = Counter(word_list)
c.most_common(100)

In [ ]:
# 토큰화된 문서
tokenized_doc = df['제목_전처리']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Gensim에서 지원하는 LDA 모델 불러오기
from gensim.models.ldamodel import LdaModel
from gensim import corpora

dictionary = corpora.Dictionary(tokenized_doc)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 단어 말뭉치(Corpus) 생성
corpus = [dictionary.doc2bow(text) for text in tokenized_doc];corpus

In [ ]:
NUM_TOPICS = 10 # 토픽 개수 지정 (하이퍼 파라미터)
ldamodel = LdaModel(corpus,
                    num_topics = NUM_TOPICS,
                    id2word = dictionary,
                    passes = 30) # 학습 횟수

topics = ldamodel.print_topics(num_words = 10) # 토픽 저장

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
topics

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.107*"보안" + 0.059*"시대" + 0.058*"우려" + 0.055*"영상" + 0.054*"기업" + 0.048*"편리" + 0.047*"양날" + 0.047*"이게" + 0.014*"머스크" + 0.013*"메타"'),
 (1,
  '0.067*"최대" + 0.066*"기자" + 0.062*"효율" + 0.059*"환자" + 0.059*"도래" + 0.058*"가리비" + 0.020*"오픈" + 0.013*"검토" + 0.010*"로봇" + 0.010*"미국"'),
 (2,
  '0.066*"구글" + 0.058*"해법" + 0.056*"제국" + 0.029*"데이터" + 0.023*"클라우드" + 0.012*"비디아" + 0.012*"사업" + 0.010*"솔루션" + 0.010*"거대" + 0.010*"기업"'),
 (3,
  '0.069*"가능" + 0.066*"사람" + 0.064*"과학" + 0.055*"포커스" + 0.053*"마비" + 0.053*"스푼" + 0.012*"시대" + 0.012*"규제" + 0.011*"적용" + 0.011*"챗GPT"'),
 (4,
  '0.047*"생각" + 0.045*"클럽" + 0.042*"반도체" + 0.022*"투자" + 0.022*"개발" + 0.019*"기술" + 0.017*"한국" + 0.017*"세계" + 0.016*"지원" + 0.015*"스타트업"'),
 (5,
  '0.058*"인공지능" + 0.035*"활용" + 0.033*"생성" + 0.031*"개최" + 0.030*"교육" + 0.019*"포럼" + 0.016*"시대" + 0.012*"미래" + 0.012*"우리" + 0.011*"칼럼"'),
 (6,
  '0.064*"소통" + 0.062*"나선" + 0.034*"정보" + 0.031*"서비스" + 0.026*"챗GPT" + 0.021*"공개" + 0.017*"개인" + 0.015*"도입" + 0.015*"활용" + 0.011*"스테이지"'),
 (7,
  

In [ ]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
